In [1]:
import dotenv
import os
from sqlalchemy import create_engine
import pandas as pd
"""
    Dal database AdventureWorks estraiamo la tabella dimproduct 
"""    

dotenv.load_dotenv(override= True)

username = os.getenv("username")
password = os.getenv("password")
host = os.getenv("host")
dbname = os.getenv("dbname")

conn_string = f"mysql+pymysql://{username}:{password}@{host}/{dbname}"
db_engine = create_engine(conn_string)
query = 'SELECT * FROM dimproduct'
query2 = 'SELECT * FROM dimcustomer'

dimproduct = pd.read_sql(query,db_engine)
dimcustomer = pd.read_sql(query2, db_engine)

db_engine.dispose()

In [9]:
"""
    • Sulla colonna DealerPrice, utilizzando il metodo .round(), 
    arrotondiamo i valori alle due cifre decimali, e poi al valore intero più vicino 
"""

dimproduct["DealerPrice"] = dimproduct["DealerPrice"].round(2)

In [10]:
dimproduct["DealerPrice"] = dimproduct["DealerPrice"].round(0)

In [11]:
"""
    • Utilizzando il metodo .clip(),facciamo in modo che i valori siano compresi tra un 
    minimo di 0 e un massimo di 1000.
"""
dimproduct["DealerPrice"] = dimproduct["DealerPrice"].clip(0,1000)


In [12]:
import numpy as np


years = 5 
a = pd.DataFrame({
                    "Mese": list("GFMAMGLASOND"*years),  
                    "Anno": np.repeat(list(range(years)), 12), 
                    "Guadagni": np.random.randint(800, 5000, 12*years)
                })

print(a.groupby("Mese")["Guadagni"].cumsum())



0      1920
1      1902
2      3521
3      3482
4      7952
5      3110
6      4799
7      7086
8      2023
9      3148
10     3601
11     2266
12     6129
13     5183
14    10965
15     8840
16    12215
17     8435
18     5829
19    12243
20     6667
21     4943
22     4610
23     3173
24    12228
25     9153
26    13569
27    14157
28    15559
29    14235
30     8797
31    16415
32     8725
33     7539
34     8275
35     4721
36    18994
37    12937
38    17757
39    19290
40    21907
41    22795
42    12057
43    20407
44    12069
45    10648
46    11039
47     8663
48    27006
49    15445
50    26737
51    24967
52    30355
53    30496
54    15950
55    29249
56    16681
57    12533
58    14869
59    12993
Name: Guadagni, dtype: int64


In [13]:
"""
    • Come sopra, ma diviso per ogni anno
"""
print(a.groupby("Anno")["Guadagni"].cumsum())


0      1920
1      3822
2      7343
3     10825
4     15256
5     16446
6     21245
7     24849
8     26872
9     30020
10    33621
11    35887
12     3019
13     6300
14     9313
15    11067
16    12317
17    14623
18    15653
19    19056
20    23700
21    25495
22    26504
23    27411
24     3793
25     7763
26     9117
27    11031
28    13021
29    15028
30    17996
31    20254
32    22312
33    24908
34    28573
35    30121
36     4759
37     8543
38    10741
39    13616
40    17766
41    21567
42    24827
43    25944
44    29288
45    32397
46    35161
47    39103
48     4211
49     6719
50    11549
51    16109
52    19727
53    23217
54    27110
55    31392
56    36004
57    37889
58    41719
59    46049
Name: Guadagni, dtype: int64


In [14]:
"""
    Da dimcustomer:


    • Trasformiamo i nomi dei clienti in modo che abbiano solo lettere minuscole,
    e i cognomi in modo che abbiano solo lettere maiuscole 
"""
dimcustomer["FirstName"] = dimcustomer["FirstName"].str.lower()
dimcustomer["LastName"] = dimcustomer["LastName"].str.upper()

print(dimcustomer.sample())


       CustomerKey  GeographyKey CustomerAlternateKey Title FirstName  \
11781        22781           352           AW00022781  None    hunter   

      MiddleName LastName  NameStyle   BirthDate MaritalStatus  ...  \
11781       None   POWELL          0  1956-12-19             M  ...   

      EnglishOccupation SpanishOccupation FrenchOccupation  HouseOwnerFlag  \
11781      Professional       Profesional            Cadre               1   

       NumberCarsOwned    AddressLine1 AddressLine2         Phone  \
11781                1  6601 Browse Ct         None  516-555-0191   

      DateFirstPurchase CommuteDistance  
11781        2014-01-07       10+ Miles  

[1 rows x 29 columns]


In [15]:
"""
    • Sulla colonna EmailAddress, utilizzando il metodo .str.split(), estraiamo nome utente e dominio 
"""
dimcustomer[["Username","Domain"]] = dimcustomer["EmailAddress"].str.split("@", expand= True)
print(dimcustomer.sample())


      CustomerKey  GeographyKey CustomerAlternateKey Title FirstName  \
2172        13172           626           AW00013172  None  isabella   

     MiddleName LastName  NameStyle   BirthDate MaritalStatus  ...  \
2172          D     COOK          0  1974-03-18             S  ...   

     FrenchOccupation HouseOwnerFlag NumberCarsOwned       AddressLine1  \
2172        Direction              0               1  5998 Hilltop Road   

      AddressLine2         Phone DateFirstPurchase CommuteDistance  \
2172          None  248-555-0151        2013-10-06       1-2 Miles   

        Username               Domain  
2172  isabella86  adventure-works.com  

[1 rows x 31 columns]


In [16]:

"""
     • Sulla colonna Phone, estraiamo ogni parte del numero 
     (ad es. da "1 (11) 500 555-0162" a ["1", "(11)", "500", "555-0162"]) 
"""  
dimcustomer[["internationaPrfx","AreaPrfx", "LocalPrfx", "PhNumber" ]] = dimcustomer["Phone"].str.split(" ", expand= True)
print(dimcustomer.sample())

      CustomerKey  GeographyKey CustomerAlternateKey Title FirstName  \
3718        14718           155           AW00014718  None   gregory   

     MiddleName LastName  NameStyle   BirthDate MaritalStatus  ...  \
3718       None     RAJI          0  1977-04-13             S  ...   

     AddressLine2                Phone DateFirstPurchase  CommuteDistance  \
3718         None  1 (11) 500 555-0185        2013-03-06        1-2 Miles   

       Username               Domain internationaPrfx AreaPrfx LocalPrfx  \
3718  gregory25  adventure-works.com                1     (11)       500   

      PhNumber  
3718  555-0185  

[1 rows x 35 columns]


In [17]:
"""    
    • Utilizzando il metodo .str.contains(), estraiamo tutti gli indirizzi e-mail che contengono il numero "21"
"""
contains_twentyone = dimcustomer[dimcustomer['EmailAddress'].str.contains('21')]
print(contains_twentyone.sample())

       CustomerKey  GeographyKey CustomerAlternateKey Title FirstName  \
13136        24136           188           AW00024136  None    alisha   

      MiddleName LastName  NameStyle   BirthDate MaritalStatus  ...  \
13136          E       HU          0  1980-12-29             S  ...   

      AddressLine2                Phone DateFirstPurchase  CommuteDistance  \
13136         None  1 (11) 500 555-0175        2013-04-02        2-5 Miles   

       Username               Domain internationaPrfx AreaPrfx LocalPrfx  \
13136  alisha21  adventure-works.com                1     (11)       500   

       PhNumber  
13136  555-0175  

[1 rows x 35 columns]


In [18]:
"""
    • Estraiamo tutti gli indirizzi e-mail che contengono il numero "20" oppure il numero "10"
"""
ten_or_twenty = dimcustomer[dimcustomer['EmailAddress'].str.contains('20|10')]
print(ten_or_twenty.sample)

<bound method NDFrame.sample of        CustomerKey  GeographyKey CustomerAlternateKey Title   FirstName  \
1            11001            37           AW00011001  None      eugene   
10           11010            22           AW00011010  None   jacquelyn   
22           11022           609           AW00011022  None       ethan   
30           11030            28           AW00011030  None     bethany   
32           11032            35           AW00011032  None      denise   
...            ...           ...                  ...   ...         ...   
18395        29395           203           AW00029395  None       lydia   
18397        29397           193           AW00029397  None      walter   
18415        29415           223           AW00029415  None       oscar   
18418        29418           163           AW00029418  None     lindsey   
18428        29428           156           AW00029428  None  kristopher   

      MiddleName LastName  NameStyle   BirthDate MaritalStatus  ...

In [20]:
"""
    • Calcolare la lunghezza di ogni indirizzo e-mail ed estrarre i dieci più lunghi e i dieci più corti
"""
dimcustomer['EmailLength'] = dimcustomer['EmailAddress'].str.len()
longest_ten = dimcustomer.nlargest(10, 'EmailLength')
shortest_ten = dimcustomer.nsmallest(10, 'EmailLength')
print(longest_ten)
print(shortest_ten)

       CustomerKey  GeographyKey CustomerAlternateKey Title    FirstName  \
979          11979            69           AW00011979  None  christopher   
1767         12767           648           AW00012767  None  christopher   
2655         13655           609           AW00013655  None  christopher   
3512         14512           633           AW00014512  None  christopher   
4772         15772           361           AW00015772  None  christopher   
4973         15973           638           AW00015973  None  christopher   
7755         18755            50           AW00018755  None  christopher   
8768         19768           310           AW00019768  None  christopher   
9242         20242            18           AW00020242  None  christopher   
10233        21233           358           AW00021233  None  christopher   

      MiddleName  LastName  NameStyle   BirthDate MaritalStatus  ...  \
979         None   JOHNSON          0  1962-11-05             M  ...   
1767           A  R

In [21]:
"""• Modificare il dominio degli indirizzi e-mail da "adventure-works.com" a "aw-db.com" mediante il metodo .str.replace()
"""
dimcustomer['EmailAddress'] = dimcustomer['EmailAddress'].str.replace('adventure-works.com', 'aw-db.com')
print(dimcustomer.sample())

      CustomerKey  GeographyKey CustomerAlternateKey Title FirstName  \
7386        18386           316           AW00018386  None     kevin   

     MiddleName LastName  NameStyle   BirthDate MaritalStatus  ...  \
7386       None     SHAN          0  1968-11-22             M  ...   

             Phone DateFirstPurchase CommuteDistance  Username  \
7386  125-555-0141        2013-12-10      5-10 Miles   kevin34   

                   Domain  internationaPrfx AreaPrfx LocalPrfx PhNumber  \
7386  adventure-works.com      125-555-0141     None      None     None   

     EmailLength  
7386          27  

[1 rows x 36 columns]


In [23]:
"""
    • Dalla colonna AddressLine1 estraiamo tutti gli indirizzi che contengono la sottostringa "Street"
"""
contains_street = dimcustomer[dimcustomer['AddressLine1'].str.contains('Street')]
print(contains_street)

       CustomerKey  GeographyKey CustomerAlternateKey Title FirstName  \
7            11007            40           AW00011007  None     marco   
12           11012           611           AW00011012  None    lauren   
32           11032            35           AW00011032  None    denise   
63           11063           634           AW00011063  None    angela   
125          11125            37           AW00011125  None      dana   
...            ...           ...                  ...   ...       ...   
18273        29273           298           AW00029273  None   timothy   
18292        29292            34           AW00029292  None     diana   
18323        29323            60           AW00029323  None    andrea   
18333        29333            27           AW00029333  None      abby   
18351        29351            49           AW00029351  None     jason   

      MiddleName   LastName  NameStyle   BirthDate MaritalStatus  ...  \
7           None      MEHTA          0  1969-11-06

In [25]:
"""
    Dai beginner_datasets carichiamo in un DataFrame il file facebook.csv, che contiene dei post con data 
    di pubblicazione, tipo (foto, video, …) e numero di reactions raccolte: 
    • Con la funzione pd.to_datetime() convertiamo la colonna status_published in formato Timestamp 
"""
facebook = pd.read_csv("facebook.csv")
facebook["status_published"] = pd.to_datetime(facebook["status_published"])
facebook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7050 entries, 0 to 7049
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   status_id         7050 non-null   object        
 1   status_type       7050 non-null   object        
 2   status_published  7050 non-null   datetime64[ns]
 3   num_reactions     7050 non-null   int64         
 4   num_comments      7050 non-null   int64         
 5   num_shares        7050 non-null   int64         
 6   num_likes         7050 non-null   int64         
 7   num_loves         7050 non-null   int64         
 8   num_wows          7050 non-null   int64         
 9   num_hahas         7050 non-null   int64         
 10  num_sads          7050 non-null   int64         
 11  num_angrys        7050 non-null   int64         
dtypes: datetime64[ns](1), int64(9), object(2)
memory usage: 661.1+ KB


In [26]:

"""
    • Utilizzando il metodo .df.strftime(), formatta le date delle pubblicazioni in un formato specifico (ad esempio, "%Y-%m-%d %H:%M:%S"). 
"""    
mod = facebook["status_published"].dt.strftime("%Y/%m/%d %H:%M:%S")
print(mod)

0       2018/04/22 06:00:00
1       2018/04/21 22:45:00
2       2018/04/21 06:17:00
3       2018/04/21 02:29:00
4       2018/04/18 03:22:00
               ...         
7045    2016/09/24 02:58:00
7046    2016/09/23 11:19:00
7047    2016/09/21 23:03:00
7048    2016/09/20 00:43:00
7049    2016/09/10 10:30:00
Name: status_published, Length: 7050, dtype: object


In [27]:
"""
    • Utilizzando gli attributi .dt.year , .dt.month , .dt.day , .dt.dayofweek , .dt.dayofyear, ottieniamo informazioni specifiche 
    sulle date delle transazioni, come l'anno, il mese, il giorno della settimana, il giorno dell'anno, ecc.
"""

anni = facebook["status_published"].dt.year
mesi = facebook["status_published"].dt.month
giorni = facebook["status_published"].dt.day
giorni_settimana = facebook["status_published"].dt.dayofweek
giorni_anno = facebook["status_published"].dt.day_of_year


In [28]:
"""    
    • Estraiamo solo i post relativi al 2012 
    
""" 
twoth_twelve = facebook[facebook["status_published"].dt.year == 2012]

In [29]:
"""
•Estraiamo solo i post relativi a maggio 2018   
"""
may_eighteen = facebook[(facebook["status_published"].dt.year == 2018) & (facebook["status_published"].dt.month == 5)]

In [30]:
"""
    • Troviamo il primo e ultimo post pubblicati in ogni anno 
"""
weekends = giorni_settimana[5] + giorni_settimana[6]
resto_settimana = giorni_settimana.sum() - weekends

In [31]:
"""
    • Utilizzando il metodo .tz_localize(), aggiungiamo un fuso orario alle date delle transazioni 
"""
facebook["status_published"] = facebook["status_published"].dt.tz_localize("UTC")
print(facebook["status_published"])

0      2018-04-22 06:00:00+00:00
1      2018-04-21 22:45:00+00:00
2      2018-04-21 06:17:00+00:00
3      2018-04-21 02:29:00+00:00
4      2018-04-18 03:22:00+00:00
                  ...           
7045   2016-09-24 02:58:00+00:00
7046   2016-09-23 11:19:00+00:00
7047   2016-09-21 23:03:00+00:00
7048   2016-09-20 00:43:00+00:00
7049   2016-09-10 10:30:00+00:00
Name: status_published, Length: 7050, dtype: datetime64[ns, UTC]


In [35]:
"""
    • Calcoliamo la differenza di tempo tra due transazioni 
"""
from random import randint
print(facebook.iloc[1]["status_published"] - facebook.iloc[randint(1,7050)]["status_published"])                                                                                                                                            
    

178 days 16:53:00


In [33]:
"""    
    • Quanti tipi di post ci sono? E quanti per ogni tipo? 
"""    
print(facebook.describe())

       num_reactions  num_comments   num_shares    num_likes    num_loves  \
count    7050.000000   7050.000000  7050.000000  7050.000000  7050.000000   
mean      230.117163    224.356028    40.022553   215.043121    12.728652   
std       462.625309    889.636820   131.599965   449.472357    39.972930   
min         0.000000      0.000000     0.000000     0.000000     0.000000   
25%        17.000000      0.000000     0.000000    17.000000     0.000000   
50%        59.500000      4.000000     0.000000    58.000000     0.000000   
75%       219.000000     23.000000     4.000000   184.750000     3.000000   
max      4710.000000  20990.000000  3424.000000  4710.000000   657.000000   

          num_wows    num_hahas     num_sads   num_angrys  
count  7050.000000  7050.000000  7050.000000  7050.000000  
mean      1.289362     0.696454     0.243688     0.113191  
std       8.719650     3.957183     1.597156     0.726812  
min       0.000000     0.000000     0.000000     0.000000  
25%   

In [ ]:
"""
    Dai beginner_dataset carichiamo in un DataFrame il file pokemon.csv: 
    • Tramite i metodi .isnull() e .sum() controlliamo se ci sono valori nulli nel dataset e 
    contiamo quanti valori nulli ci sono in ogni colonna 
    • Ci sono valori nulli? 
    • Eliminiamo le righe che contengono valori nulli
"""

pokemon = pd.read_csv("pokemon.csv")

print(pokemon.isnull().count())

pokemon.dropna(inplace=True)

print(pokemon.isnull().count())

In [37]:
"""
    Dai beginner_dataset carichiamo in un DataFrame il file automobile.csv:
    • Ci sono valori nulli? Dove? Quanti? 
"""
auto = pd.read_csv("automobile.csv")

print(auto.isnull().sum())


symboling             0
normalized-losses    37
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  0
stroke                0
compression-ratio     0
horsepower            0
peak-rpm              0
city-mpg              0
highway-mpg           0
price                 0
dtype: int64


In [38]:
"""
    • Quali righe hanno un valore nullo nella colonna num-of-doors? 
"""    
print(auto[auto['num-of-doors'].isnull()].index)

Index([26, 60], dtype='int64')


In [53]:
"""
    Abbiamo il seguente DataFrame che raccoglie le misurazioni di un sensore che misura la temperatura atmosferica giornaliera:
"""
import numpy as np, pandas as pd 
temp = pd.DataFrame({
                        "Giorno": [0, 1, 2, 3, 4, 5, 6 ,7, 8, 9, 10, 11, 12],       
                        "Temperature": [18, 19, 18, np.nan, 21, 20, 20, np.nan, 21, 23, np.nan, 23, 24]
                    }) 
"""
        • Il sensore a volte non funziona, dunque alcuni dati sono mancanti: quale sarebbe la migliore strategia per gestirli? 
"""



'\n        • Il sensore a volte non funziona, dunque alcuni dati sono mancanti: quale sarebbe la migliore strategia per gestirli? \n'

In [54]:
media = temp["Temperature"].mean()
temp = temp['Temperature'].fillna(media)
print(temp)

0     18.0
1     19.0
2     18.0
3     20.7
4     21.0
5     20.0
6     20.0
7     20.7
8     21.0
9     23.0
10    20.7
11    23.0
12    24.0
Name: Temperature, dtype: float64
